## Script Name: Step1-Gr-23-Vdo-audo-extraction_ocring
##### This program takes basepath of a folder having AD-Videos and process each video to extract video frames, audio and then further processes the extracted files to get the annoted text from video frames and audio files.

#### This Program does the following steps:
##### Step 1. Frames extraction from video at the interval of one second
##### Step 2. Audio extraction from video for performing speect to text conversion
##### Step 3. Converting extracted frames to GrayScale for better results during OCR
##### Step 4. Text extraction from images using OCR

In [1]:
# Install all required libraries

In [2]:
# Import all libraries

import sys
import os, os.path
import numpy  as np
import cv2  #For image processing
import moviepy.editor as mp  #For Audio
from pydub import AudioSegment # For Audio extract 
import glob, os, errno
import requests
import pandas as pd
# For OCR
import pytesseract
from pytesseract import Output
#from PIL import Image
import re  
import wx  # For wwidth & height

C:\Users\Asus\Anaconda3\lib\site-packages\numpy\_distributor_init.py:32: UserWarning: loaded more than 1 DLL from .libs:
C:\Users\Asus\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.NOIJJG62EMASZI6NYURL6JBKM4EVBGM7.gfortran-win_amd64.dll
C:\Users\Asus\Anaconda3\lib\site-packages\numpy\.libs\libopenblas.PYQHXLVVQ7VESDPUVUADXEVJOBGHJPAY.gfortran-win_amd64.dll
  stacklevel=1)


In [3]:
a=50
frame= ("%0.3d"% (a))

In [4]:
frame="{0:03}".format(a)

In [5]:
print(frame)

050


In [6]:
def width_and_height_calculator_in_pixel(txt, fontname, fontsize):
    app=[]; app = wx.App(None)
    dc   = wx.ScreenDC()
   #dc.SetFont(...) # todo: https://wxpython.org/Phoenix/docs/html/wx.DC.html#wx.DC.SetFont
    size = dc.GetTextExtent(txt)
    return size

#import xml.etree.ElementTree as XT
#from threading import Thread
#import time
print ("OpenCV version: " + cv2.__version__)

def checknonAscii(AnyString):
    for c in AnyString:
      if 0 <= ord(c) <= 127:
         return True
      else:
         return False
     
def wordcount(value):
    # Find all non-whitespace patterns.
    list = re.findall("(\S+)", value)
    # Return length of resulting list.
    return len(list)


def string_length(str1):
    count = 0
    for char in str1:
        count += 1
    return count





OpenCV version: 4.4.0


### MoviePy (full documentation) is a Python library for video editing: cutting, concatenations, title insertions, video compositing (a.k.a. non-linear editing), video processing, and creation of custom effects.

##### pip install moviepy    - Take time
##### pip install pydub


### Provide the basepath of the folder having Ad-Videos

In [7]:
# Folder path where all input video files are present
root_folder = "C:\\Users\\Asus\\bits-capstone\\1-Next assignment\\Group-99-InVDO"

In [8]:
# Video processing and frame extraction function
# Supported video file extension: 'mp4' & 'avi'
# Output images: '.jpg' format
# Output audio files: '.wav' format

#------------------

# Input Parameters:
#  imageDir: Folder path having all input video files
#  OutimageDir: Output Folder path for extracted video frames
#  OutAudioFolder: Output Folder for normalized audio file
#  audio: Output Folder for extracted audio file

def vdo_Framing_Process(imageDir,OutimageDir,OutAudioFolder,audio):    
#------------------
    suffix = '.jpg'
    seq = 0
    #  valid extensions of base video files
    image_path_list = []
    valid_vdo_extensions = [".mp4" , "avi"] #specify all valid extensions here
    valid_vdo_extensions = [item.lower() for item in valid_vdo_extensions]
     
    #create a list all files in directory and
    #append files with a vaild extention to image_path_list
    for file in os.listdir(imageDir):
        extension = os.path.splitext(file)[1]
        if extension.lower() not in valid_vdo_extensions:
            continue
        image_path_list.append(os.path.join(imageDir, file)) #<root_folder>/VDO-AD/VDO-AD-200.mp4        
        
     
    #loop through image_path_list to open each image
    for imagePath in image_path_list:
            seq = seq + 1
            
            temp = os.path.splitext(imagePath)[0]  
            print('imagePath:', imagePath)
            print('temp:',temp)
            # base filename for extracted images
            base_filename = os.path.basename(temp)+suffix 

            # Extract Speech from AD vido Clip
            audio_suffix = '.wav'
            # Extract audio using MoviePy Editor
            clip = mp.VideoFileClip(imagePath)
            # filename for extracted audio
            AudioFile = os.path.basename(temp)+audio_suffix
            
            # Full path of extracted audio file
            AudioFileName1 = os.path.join(os.path.dirname(imageDir),audio,AudioFile)            
            # print(AudioFileName1)
            clip.audio.write_audiofile(AudioFileName1)
            
            # Call close_clip function
            close_clip(clip)

            # Normalized Audio
            audio_suffix = '.wav'
            # Extraction using Pydub library
            sound = AudioSegment.from_file(AudioFileName1, "wav")
            
            
            #------------------------------------------------------
            # Call match_target_amplitude function
            # an audio file to be the same average amplitude, 
            # choosing an average amplitude (-20 dBFS)
            normalized_sound = match_target_amplitude(sound, -20.0)
            #------------------------------------------------------
            
            
            #Change of file from stereo to mono
            normalized_sound = normalized_sound.set_channels(1)    
            # print(normalized_sound)        
            
            #-------------------------------------
            AudioFileName = os.path.join(os.path.dirname(imageDir),OutAudioFolder,AudioFile)
            #AudioFileName = os.path.basename(temp)+audio_suffix
            #AudioFileName = os.path.join(OutAudioFolder+"/"+AudioFileName)
            normalized_sound.export(AudioFileName, format="wav")

            # Extract audio from AD Clip -----------------------
        
            OutimageDir = os.path.join(os.path.dirname(imageDir),OutimageDir)
            # Call image extraction function and provide input and output directory path
            extractImages(imagePath, OutimageDir,base_filename,seq)
            #image = cv2.imread(imagePath,0)      

In [9]:
# achieve normalization of peak volume

#Input Parameters:
#  sound: input audio file
#  target_dBFS: target dBFS for normalization

def match_target_amplitude(sound, target_dBFS):
    change_in_dBFS = target_dBFS - sound.dBFS
    return sound.apply_gain(change_in_dBFS)


In [10]:
# Terminate the object having the clip

#Input Parameters:
# clip: input video file for audio extraction
def close_clip(clip):
        # https://github.com/Zulko/moviepy/issues/164
        # This function required to handle 
        #OSError: [WinError 6] The handle is invalid” in VideoFileClip function
        try:
                clip.reader.close()
                del clip.reader
                if clip.audio != None:
                        clip.audio.reader.close_proc()
                        del clip.audio
                del clip
        except Exception as e:
                sys.exc_clear()  


In [11]:
# extractImages: Process video file and extract images

# Input Parameters:
#  pathIn: folder path having video files
#  pathOutfolder path for extracted images
#  base_filename: basefilename format for output frames
#  Vdo_Seq: frame sequence number

def extractImages(pathIn, pathOut,base_filename,Vdo_Seq):
    count = 0
    
    # capture the video
    vidcap = cv2.VideoCapture(pathIn)
    success,image = vidcap.read()
    success = True
    w_base_filename = ""
    suffix = '.jpg'
    while success:
      # cap.set(cv2.CAP_PROP_POS_MSEC,1000) is responsible for skipping 
      # directly to the 1st second in the video (1000th millisecond).  
      # extract frame every one second
      vidcap.set(cv2.CAP_PROP_POS_MSEC,(count*1000))
      success,image = vidcap.read()
      
    
      ## Stop when last frame is identified
      image_last = cv2.imread("frame{}.png".format(count-1))
      if np.array_equal(image,image_last):
          break
      #print ('Read a new frame: ', success)
   
      if w_base_filename!= base_filename:
         temp = os.path.splitext(base_filename)[0]
         
         pathOutFinal = pathOut + "//"+ temp
         #print ('pathOutFinal: ', pathOutFinal)
         if not os.path.exists(pathOutFinal):
            os.makedirs(pathOutFinal)
            w_base_filename = base_filename
      #print ('base_filename0: ', base_filename)
      #Final_filename of the extracted frame image
      count_str="{0:03}".format(count)  
      Final_filename = "/"+temp+"_Capstone_Gr_23_"+ str(Vdo_Seq)+"Frame"+ count_str+ suffix
      #print ('base_filename1: ', base_filename)
      
      Outimage = os.path.join(pathOutFinal+Final_filename)
      #print ('Outimage: ', Outimage)
    
      cv2.imwrite(Outimage, image)   # save frame as JPEG file
      count = count + 1  

In [12]:
def OCRprep(ScaledimageDir,OCRTEXTfolder):
    #------------------

    CSV_dict = []
    df = pd.DataFrame()
    
    
    suffix = '.jpg'
    base_filename='dummy'

    #  valid extensions of base image
    image_path_list = []
    valid_image_extensions = [".jpg", ".jpeg", ".png", ".tif", ".tiff"] #specify your vald extensions here
    valid_image_extensions = [item.lower() for item in valid_image_extensions]
     
    #create a list all files in directory and
    #append files with a vaild extention to image_path_list
    for file in os.listdir(ScaledimageDir):
        extension = os.path.splitext(file)[1]
        if extension.lower() not in valid_image_extensions:
            continue
        image_path_list.append(os.path.join(ScaledimageDir, file))
    
    
    
    #loop through image_path_list to open each image
    for imagePath in image_path_list:
        
        image = cv2.imread(imagePath)
                     
            #'''
            # Call OCR_Tesseract function for each sclaed image
            #'''
        CSV_dict_file = OCR_process(imagePath , OCRTEXTfolder)
        print(CSV_dict_file)
        if  len(CSV_dict_file)==0:
            continue
        CSV_dict= CSV_dict + CSV_dict_file

                    
        for value in CSV_dict:
            print (value) 

    Newcol_names = ["LineNO","FileName" ,"Text","WordWidth" , "Wordheight" ] 
    df_OCRText = pd.DataFrame.from_records(CSV_dict, columns=Newcol_names)
    return df_OCRText

    #fileName = vdo_filename+".csv"
    #df_OCRText.to_csv("C:\\Users\\Asus\\bits-capstone\\1-Next assignment\\Group-99-InVDO\\VDOFrames\\VDO-AD-105\\OCRtext\\" +fileName+ "_ocrText.csv", sep=',' , index=False)



In [13]:
import easyocr
### 0) Easy OCR should be tried on Colab - Based on Pytorch
### 1) Keras OCR (Returns the list) - Next Priority
### Should be tried on Google Colab
### 2) Google API---> This will return in Json file
### 3) Speech to Text --> Code snippet will be provided
     - there are many options (Python Package)
     - Speech Recognition (Garima tried) - Try Hyper Parameter Tuning
     - Try finding the basic Architecture 
### 4) Object Detection 
        - Keras Yolo
        - https://github.com/OlafenwaMoses/ImageAI
  

In [14]:
def OCR_process(scaledImage , OCRTEXTfolder ): 
    
    easyOcr = True

    # Extract file name from path, no matter 
    # what the os/path format
    filefullName = os.path.basename(scaledImage)
    fileName = os.path.splitext(filefullName)[0]
    
    #splitFilePath = scaledImage.rsplit ('\\', 2)
    #VDOName = splitFilePath [0]
    
    tessdata_dir_config =  "C:\\Users\\Asus\\bits-capstone\\1-Next assignment\\Group-99-InVDO"
    img = cv2.imread(scaledImage)
    print(img.shape)
    total_text =""
    if easyOcr == False:   
        d = pytesseract.image_to_data(img, output_type=Output.DICT,config=tessdata_dir_config)
        total_text = pytesseract.image_to_string(img, lang='eng', config=tessdata_dir_config) 
    else:
        reader = easyocr.Reader(['en']) 
        results = reader.readtext(img)
        for (bbox, text, prob) in results:
        # display the OCR'd text and associated probability
            print("[INFO] {:.4f}: {}".format(prob, text))
            # unpack the bounding box
            (tl, tr, br, bl) = bbox
            tl = (int(tl[0]), int(tl[1]))
            tr = (int(tr[0]), int(tr[1]))
            br = (int(br[0]), int(br[1]))
            bl = (int(bl[0]), int(bl[1]))
            # cleanup the text and draw the box surrounding the text along
            # with the OCR'd text itself
            text = "".join([c if ord(c) < 128 else "" for c in text]).strip()
            total_text = total_text + '\n'+text 
        
    # write to a text file
    with open('Output.txt', 'w',5 ,'utf-8') as text_file:
      text_file.write(total_text)  # Temporary write
    file1 = open('Output.txt', 'r', encoding="utf8") 
    Lines = file1.readlines() 
    CSV_dict = []
    count = 1                
    # Strips the newline character 
    for line in Lines: 
        if re.match(r'^\s*$', line): # Check empty line
           continue
        line = line.strip()
        if (easyOcr == False):
            text_size = width_and_height_calculator_in_pixel(line, "Calibri", 11)
            word_width= text_size[0]
            word_height= text_size[1]

        data_dict = []

        data_dict.append(count)
        data_dict.append(fileName)
        data_dict.append(line)  # Text 
        data_dict.append(word_width)
        data_dict.append(word_height)
       
        
        CSV_dict.append(data_dict)
        # if data_dict list empty - bypass
        if not CSV_dict:
            continue
  
        
        #print("Line{}, {}, {}, {}, {}".format(count,fileName, line ,word_width, word_height))
        count = count +1
    Newcol_names = ["LineNO","FileName" ,"Text","WordWidth" , "Wordheight" ] 
    #print(CSV_dict)
    #df_OCRText = pd.DataFrame.from_records(CSV_dict, columns=Newcol_names)
    #df_OCRText = pd.DataFrame.from_records(CSV_dict)
    #df.append(df_OCRText)
    #df_OCRText.head()
    #print("###### I am Here ---")
    #del(df_OCRText)
    
    #fileName = vdo_filename+".csv"
    #df_OCRText.to_csv("C:\\Users\\Asus\\bits-capstone\\1-Next assignment\\Group-99-InVDO\\VDOFrames\\VDO-AD-105\\OCRtext\\" +fileName+ "_ocrText.csv", sep=',' , index=False)

    return CSV_dict




In [15]:
scaledFolder = "C:\\Users\\Asus\\bits-capstone\\1-Next assignment\\Group-99-InVDO\\VDOFrames\\VDO-AD-105\\ScaledImage"

In [16]:
ocrFolder = "C:\\Users\\Asus\\bits-capstone\\1-Next assignment\\Group-99-InVDO\\VDOFrames\\VDO-AD-105\\OCRtext"

In [17]:
# root_folder - basepath of folder having all input video files
print(root_folder)

# Set folder path for extracted video frames
FramesVdo  = root_folder+"\\VDOFrames"
# Set folder path for normalized audio
FullAudio  = root_folder+"\\FullAudio"
# Set folder path for audio extracted from Ad video clip
audio = root_folder+"\\Audio"

# Create above folders if not exists already
if not os.path.exists(FramesVdo):
   os.makedirs(FramesVdo) 

if not os.path.exists(audio):
   os.makedirs(audio) 
   
if not os.path.exists(FullAudio):
   os.makedirs(FullAudio)    


C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO


### Step 1. Frames extraction from video at the interval of one second
### Step 2. Audio extraction from video for performing speect to text conversion


In [18]:
# Iterate through each folder in basepath folder & do not process video frames folder
for child in os.listdir(root_folder):
    if child == "VDOFrames":
        continue
    print(child)
    baseimagefolders = os.path.join(root_folder, child)
    print(baseimagefolders)
    if os.path.isdir(baseimagefolders):
        outImage  = 'VDOFrames'
        AudioFolder = 'FullAudio'
        audio = 'Audio'
#         print("--- ----------------------------------------")    
#         print('baseimagefolders:',baseimagefolders)
#         print('outImage:',outImage)
#         print('AudioFolder:',AudioFolder)
#         print('audio:',audio)
        vdo_Framing_Process(baseimagefolders,outImage,AudioFolder,audio)
       
print("------------------------Processing Completed------------------------")

Audio
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\Audio
FullAudio
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\FullAudio
VDO-AD
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD
imagePath: C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD\VDO-AD-105.mp4
temp: C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD\VDO-AD-105


chunk:   0%|▎                                                                | 3/659 [00:00<00:33, 19.47it/s, now=None]

MoviePy - Writing audio in C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\Audio\VDO-AD-105.wav


MoviePy - Done.
imagePath: C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD\VDO-AD-200.mp4
temp: C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD\VDO-AD-200


chunk:   0%|▎                                                                | 3/661 [00:00<00:34, 19.07it/s, now=None]

MoviePy - Writing audio in C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\Audio\VDO-AD-200.wav


MoviePy - Done.
VDO-AD-105.mp4
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD-105.mp4
VDO-AD-200.mp4
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDO-AD-200.mp4
------------------------Processing Completed------------------------


### Step 3. Converting extracted frames to GrayScale for better results during OCR
### Step 4. Text extraction from images using OCR

In [19]:
# Set Video Frames directory
vdoFramesDir = os.path.join(root_folder, 'VDOFrames')
print(vdoFramesDir)

# Iterate through each file, i.e. extracted video frame image
# Convert to grayscaled image and perform OCRing on the scaled image
for child in os.listdir(vdoFramesDir):
    childDir = os.path.join(vdoFramesDir,child)
    print(vdoFramesDir)
    print(childDir)
    # create gray image folder
    grayImageDir = os.path.join(childDir, 'GrayImage')
    if not os.path.exists(grayImageDir):
        os.makedirs(grayImageDir)
        
    # create scaled image folder
    scaledImageDir = os.path.join(childDir, 'ScaledImage')
    if not os.path.exists(scaledImageDir):
        os.makedirs(scaledImageDir)
        
    # create ocrtext folder
    ocrTextDir = os.path.join(childDir, 'OCRtext')
    if not os.path.exists(ocrTextDir):
        os.makedirs(ocrTextDir)

    # change current directory to child directory
    os.chdir(childDir)
    for fil in glob.glob("*.jpg"):
        image = cv2.imread(fil) 
        
        # grayimage conversion
        gray_image = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY) # convert to greyscale
        cv2.imwrite(os.path.join(grayImageDir,fil),gray_image) # write to location with same name
        
        # scaled image conversion
        height, width, channels = image.shape
        scaled_image = image
        if height * width > 2120 * 1590:
            scaled_image = cv2.resize(image, (1200, 1200))
        cv2.imwrite(os.path.join(scaledImageDir,fil),scaled_image) # write to location with same name
        
    
    
    scaledImageDir = os.path.join(childDir, 'ScaledImage')
    ocrFolder = os.path.join(childDir, 'OCRtext')
    ocrDataFrame = OCRprep(scaledImageDir, ocrFolder)
    basename = os.path.basename(childDir)
    
    ocrDataFrame.to_csv(ocrFolder +basename+ "_ocrText.csv", sep=',' , index=False)
        # Extract text from image using OCRing
        # ......
        # ......
        # IN PROGRESS
        # ......
        # ......
        

C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDOFrames
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDOFrames
C:\Users\Asus\bits-capstone\1-Next assignment\Group-99-InVDO\VDOFrames\VDO-AD-105
(1080, 1920, 3)
[]
(1080, 1920, 3)
[INFO] 0.3167: 3
[INFO] 0.9513: 8%
[[1, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '3', 8, 16], [2, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '8%', 19, 16]]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '3', 8, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '8%', 19, 16]
(1080, 1920, 3)
[INFO] 0.0000: 07==4_165=={=t_{=3_]
[INFO] 0.0001: $5e#1_53
[INFO] 0.0000: 0152
[INFO] 0.0000: i: (15set(;_37["4[=
[INFO] 0.0094: narlcel="
[INFO] 0.0004: T1te7
[[1, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '07==4_165=={=t_{=3_]', 146, 16], [2, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '$5e#1_53', 64, 16], [3, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '0152', 32, 16], [4, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'i: (15set(;_37["4[=', 114, 16], [5, 'VDO-AD-105_Capstone

[INFO] 0.0007: urLtr
[INFO] 0.0772: fuelcelleneigy
[INFO] 0.5781: fuelcelleneigy
[[1, 'VDO-AD-105_Capstone_Gr_23_1Frame012', 'urLtr', 40, 20], [2, 'VDO-AD-105_Capstone_Gr_23_1Frame012', 'fuelcelleneigy', 91, 16], [3, 'VDO-AD-105_Capstone_Gr_23_1Frame012', 'fuelcelleneigy', 91, 16]]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '3', 8, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '8%', 19, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '07==4_165=={=t_{=3_]', 146, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '$5e#1_53', 64, 16]
[3, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '0152', 32, 16]
[4, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'i: (15set(;_37["4[=', 114, 16]
[5, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'narlcel="', 58, 16]
[6, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'T1te7', 36, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame004', 'O', 10, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame006', 'Hl', 14, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame007', 'a', 10, 20]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame00

[INFO] 0.0239: NTURA GIS
[INFO] 0.0142: PoNIR PANT
[INFO] 0.0018: luull
[INFO] 0.6873: STEAM
[INFO] 0.5885: FUEL
[INFO] 0.7120: STEAM
[INFO] 0.8825: ELECTRICITY
[INFO] 0.7231: GAS
[INFO] 0.1382: CElLS
[INFO] 0.9328: GENERATION
[INFO] 0.7242: TURBINE
[INFO] 0.7942: TURBINE
[[1, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'NTURA GIS', 91, 20], [2, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'PoNIR PANT', 80, 16], [3, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'luull', 28, 16], [4, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'STEAM', 46, 16], [5, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'FUEL', 35, 16], [6, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'STEAM', 46, 16], [7, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'ELECTRICITY', 88, 16], [8, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'GAS', 27, 16], [9, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'CElLS', 39, 16], [10, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'GENERATION', 88, 16], [11, 'VDO-AD-105_Capstone_Gr_23_1Frame017', 'TURBINE', 61, 16], [12, 'VDO-AD-105_Capstone_Gr_23_1F

[INFO] 0.4221: CO2
[INFO] 0.2911: CO_
[INFO] 0.3706: co
[INFO] 0.3308: CO
[INFO] 0.4145: CO2
[INFO] 0.1038: co
[[1, 'VDO-AD-105_Capstone_Gr_23_1Frame019', 'CO2', 34, 20], [2, 'VDO-AD-105_Capstone_Gr_23_1Frame019', 'CO_', 27, 16], [3, 'VDO-AD-105_Capstone_Gr_23_1Frame019', 'co', 15, 16], [4, 'VDO-AD-105_Capstone_Gr_23_1Frame019', 'CO', 19, 16], [5, 'VDO-AD-105_Capstone_Gr_23_1Frame019', 'CO2', 27, 16], [6, 'VDO-AD-105_Capstone_Gr_23_1Frame019', 'co', 15, 16]]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '3', 8, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '8%', 19, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '07==4_165=={=t_{=3_]', 146, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '$5e#1_53', 64, 16]
[3, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '0152', 32, 16]
[4, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'i: (15set(;_37["4[=', 114, 16]
[5, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'narlcel="', 58, 16]
[6, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'T1te7', 36, 16]
[1, 'VDO-AD-105_Capstone_Gr_

[]
(1080, 1920, 3)
[]
(1080, 1920, 3)
[INFO] 0.1121: 
[INFO] 0.1534: =
[[1, 'VDO-AD-105_Capstone_Gr_23_1Frame023', '=', 9, 20]]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '3', 8, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame001', '8%', 19, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '07==4_165=={=t_{=3_]', 146, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '$5e#1_53', 64, 16]
[3, 'VDO-AD-105_Capstone_Gr_23_1Frame002', '0152', 32, 16]
[4, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'i: (15set(;_37["4[=', 114, 16]
[5, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'narlcel="', 58, 16]
[6, 'VDO-AD-105_Capstone_Gr_23_1Frame002', 'T1te7', 36, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame004', 'O', 10, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame006', 'Hl', 14, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame007', 'a', 10, 20]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame008', ':', 4, 16]
[2, 'VDO-AD-105_Capstone_Gr_23_1Frame008', '^', 5, 16]
[1, 'VDO-AD-105_Capstone_Gr_23_1Frame009', ':', 4, 20]
[2, 'VDO-AD-105_Capstone_Gr_2

...

[INFO] 0.9731: STRONG
[INFO] 0.8619: STRONG
[INFO] 0.3714: & Bright
[INFO] 0.6858: & Bright
[INFO] 0.5271: ENAMEL
[INFO] 0.7555: ENAMEL
[[1, 'VDO-AD-200_Capstone_Gr_23_2Frame025', 'STRONG', 57, 16], [2, 'VDO-AD-200_Capstone_Gr_23_2Frame025', 'STRONG', 57, 16], [3, 'VDO-AD-200_Capstone_Gr_23_2Frame025', '& Bright', 52, 16], [4, 'VDO-AD-200_Capstone_Gr_23_2Frame025', '& Bright', 52, 16], [5, 'VDO-AD-200_Capstone_Gr_23_2Frame025', 'ENAMEL', 56, 16], [6, 'VDO-AD-200_Capstone_Gr_23_2Frame025', 'ENAMEL', 56, 16]]
[1, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'Zzz', 26, 20]
[2, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'NyCuil`', 48, 16]
[3, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'From', 33, 16]
[4, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'the Makers of', 88, 16]
[5, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'fcKS', 29, 16]
[6, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'ZzzQuil', 51, 16]
[7, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'HELPS YOU FALL', 115, 16]
[8, 'VDO-AD-200_Capstone_Gr_23_2Frame000', 'ASLEE

[INFO] 0.1545: SLNSODYIL`
[INFO] 0.1536: SFNSODYNE`
[INFO] 0.9152: PRO NAMEL
[INFO] 0.5339: PI?O NAMEL
[INFO] 0.9537: STRONG
[INFO] 0.9598: STRONG
[INFO] 0.6146: & Bright
[INFO] 0.4427: & Bright
[INFO] 0.4827: ENAMEL
[INFO] 0.6015: ENAMEL
[INFO] 0.0000: [0011G4S15TDH5FH5II5
[INFO] 0.0000: rllt7^41#7:14 90!4Hvy1
[INFO] 0.0000: N0 CAHIT7 FHEVEuliO+
[INFO] 0.0000: ADIA!IIVI"#744TI)
[INFO] 0.0797: STnONC: EMAMAEI
[INFO] 0.1008: hiHOHG EMAMEL
[INFO] 0.0375: MlTEH TEEIH
[INFO] 0.2392: WHITEH TEETH"
[INFO] 0.2047: MAIHT
[INFO] 0.1731: *vs Pronamel@ Daily Protecton
[INFO] 0.3737: Pronamel com
[[1, 'VDO-AD-200_Capstone_Gr_23_2Frame026', 'SLNSODYIL`', 83, 16], [2, 'VDO-AD-200_Capstone_Gr_23_2Frame026', 'SFNSODYNE`', 90, 16], [3, 'VDO-AD-200_Capstone_Gr_23_2Frame026', 'PRO NAMEL', 80, 16], [4, 'VDO-AD-200_Capstone_Gr_23_2Frame026', 'PI?O NAMEL', 82, 16], [5, 'VDO-AD-200_Capstone_Gr_23_2Frame026', 'STRONG', 57, 16], [6, 'VDO-AD-200_Capstone_Gr_23_2Frame026', 'STRONG', 57, 16], [7, 'VDO-AD-200_Caps

[INFO] 0.0831: SFNSODYNE
[INFO] 0.2248: SFNSODYNE
[INFO] 0.6717: PRO NAMEL
[INFO] 0.5808: PRO NAMEL
[INFO] 0.7289: STRONG
[INFO] 0.9714: STRONG
[INFO] 0.4183: & Bright
[INFO] 0.3199: & Bright
[INFO] 0.6492: STRONG ENAMEL
[INFO] 0.7541: ENAMEL
[INFO] 0.5228: EMAMEL
[INFO] 0.0000: 1001HI!4SIEV0l S[MSIIIVg
[INFO] 0.0000: T2DITF#5TET0H1 5E}!5!TIVE
[INFO] 0.0000: TEEIHAMD CAHIIYIIIEVLIIIICV
[INFO] 0.0000: LzlHAIID CIMIV PHEHEIITIO"
[INFO] 0.1211: STHONC ENAIAEL
[INFO] 0.0516: SHONG EHAMEL
[INFO] 0.1326: WHITEH TEETII
[INFO] 0.0108: MITEI TEETH
[INFO] 0.3021: MINT
[INFO] 0.4616: *vs Pronamel@ Daily Protection
[INFO] 0.6249: Pronamel com
[[1, 'VDO-AD-200_Capstone_Gr_23_2Frame027', 'SFNSODYNE', 85, 16], [2, 'VDO-AD-200_Capstone_Gr_23_2Frame027', 'SFNSODYNE', 85, 16], [3, 'VDO-AD-200_Capstone_Gr_23_2Frame027', 'PRO NAMEL', 80, 16], [4, 'VDO-AD-200_Capstone_Gr_23_2Frame027', 'PRO NAMEL', 80, 16], [5, 'VDO-AD-200_Capstone_Gr_23_2Frame027', 'STRONG', 57, 16], [6, 'VDO-AD-200_Capstone_Gr_23_2Frame0

[INFO] 0.3869: SENSODYNE"
[INFO] 0.5028: SENSODYNE'
[INFO] 0.6273: PRO NAMEL
[INFO] 0.3392: PRO NAMEL
[INFO] 0.6436: STRONG
[INFO] 0.8832: STRONG
[INFO] 0.4973: & Bright
[INFO] 0.2492: & Bright
[INFO] 0.7631: STRONG ENAMEL
[INFO] 0.8370: ENAMEL
[INFO] 0.6014: ENAMEL
[INFO] 0.5366: WHITER TEETH*
[INFO] 0.1650: TOOTHPASTE FOR SENSITIVE
[INFO] 0.0339: TOOIHPASIE FOH SEISITIVE
[INFO] 0.0679: TEETH AND CAVITV PPEVENTION
[INFO] 0.0280: TEEIHAND CAVITV PHEVEIVTION
[INFO] 0.2996: STRONG ENAMEL
[INFO] 0.5436: STRONG ENAMEL
[INFO] 0.1299: WHITER TEETH
[INFO] 0.1144: MIITER TEETH
[INFO] 0.2096: HIMT
[INFO] 0.8549: MINT
[INFO] 0.1549: *vs Pronamel@ Daily Protecton
[INFO] 0.4408: Pronamel com
[[1, 'VDO-AD-200_Capstone_Gr_23_2Frame028', 'SENSODYNE"', 92, 16], [2, 'VDO-AD-200_Capstone_Gr_23_2Frame028', "SENSODYNE'", 90, 16], [3, 'VDO-AD-200_Capstone_Gr_23_2Frame028', 'PRO NAMEL', 80, 16], [4, 'VDO-AD-200_Capstone_Gr_23_2Frame028', 'PRO NAMEL', 80, 16], [5, 'VDO-AD-200_Capstone_Gr_23_2Frame028', 'STRO

[INFO] 0.9594: gsk
[INFO] 0.3812: SENSODYNE
[INFO] 0.4029: SENSODYNE'
[INFO] 0.6193: PRONAMEL
[INFO] 0.6842: PRO NAMEL
[INFO] 0.7960: STRONG
[INFO] 0.9395: STRONG
[INFO] 0.3174: & Bright
[INFO] 0.2715: & Bright
[INFO] 0.4629: STRONG ENAMEL
[INFO] 0.9482: ENAMEL
[INFO] 0.6983: ENAMEL
[INFO] 0.6150: WHITER TEETH*
[INFO] 0.2460: TOOTHPASTE FOH SENSITIVE
[INFO] 0.1833: TOOTHPASTE FOH SENSITIVE
[INFO] 0.0782: IEETH AND CAVITV PHEVENTION
[INFO] 0.0151: TEETH AND CAVITV PAEVEMTION
[INFO] 0.2851: STRONG ENAMEL
[INFO] 0.7801: STRONG ENAMEL
[INFO] 0.1813: WHITER TEETH*
[INFO] 0.1139: IHITER TEETH*
[INFO] 0.2768: HINT
[INFO] 0.5279: MINT
[INFO] 0.1826: *vs Pronamel@ Daily Protecton
[INFO] 0.3288: Pronamel com
[[1, 'VDO-AD-200_Capstone_Gr_23_2Frame029', 'gsk', 23, 16], [2, 'VDO-AD-200_Capstone_Gr_23_2Frame029', 'SENSODYNE', 86, 16], [3, 'VDO-AD-200_Capstone_Gr_23_2Frame029', "SENSODYNE'", 90, 16], [4, 'VDO-AD-200_Capstone_Gr_23_2Frame029', 'PRONAMEL', 76, 16], [5, 'VDO-AD-200_Capstone_Gr_23_2Frame

In [ ]:
pip install speechrecognition

In [1]:
import os, os.path
import cv2
#import csv
import pandas as pd
import numpy  as np
import re


import speech_recognition as sr
print ("speech_recognition version: " + sr.__version__)

speech_recognition version: 3.8.1


In [2]:
r = sr.Recognizer()

In [3]:
def ExtractSpeeech(baseAudiofolders,audioPath,base_filename,vdo_filename,SpeechText):
    
    AudioRecog = sr.AudioFile(audioPath)
    
    with AudioRecog as source:
       #r.adjust_for_ambient_noise(source)
       audio = r.record(source)
    SpeechTxt = r.recognize_google(audio, language='en-US')
    
    return SpeechTxt

In [5]:
root_folder = "C:\\Users\\Administrator\\Group-23-InVDO"
SpeechText  = root_folder+"\\SpeechTextn"  # Where all speech text stored

if not os.path.exists(SpeechText):
   os.makedirs(SpeechText) 
   
# Point  baseAudiofolders to FullAudio folder where all Audios are 
baseAudiofolders = root_folder+ "/FullAudio"


suffix = '.wav'
seq = 0
    #  valid extensions of base image
audio_path_list = []
valid_audio_extensions = [".wav"] #specify your vald extensions here
valid_audio_extensions = [item.lower() for item in valid_audio_extensions]
for file in os.listdir(baseAudiofolders):
        extension = os.path.splitext(file)[1]
        if extension.lower() not in valid_audio_extensions:
            continue
        audio_path_list.append(os.path.join(baseAudiofolders, file))
CSV_dict = []       
for audioPath in audio_path_list:
            seq = seq + 1
            print(audioPath)
            #prefix = os.path.basename(os.path.normpath(imageDir)) # Last folder of the path
            #suffix = '.jpg'  # suffix of output grey images
            temp = os.path.splitext(audioPath)[0]  
            base_filename = os.path.basename(temp)+suffix 
            vdo_filename = os.path.basename(temp)
            

            SpeechTxt = ExtractSpeeech(baseAudiofolders,audioPath,base_filename,vdo_filename,SpeechText) 
            print(SpeechTxt)
            
            # data output section 
            data_dict = []
            vdofileonly = os.path.splitext(vdo_filename)[0] 
            #data_dict.append(vdo_filename)
            data_dict.append(vdofileonly)
            data_dict.append(base_filename)
            data_dict.append(SpeechTxt)
     
        
            CSV_dict.append(data_dict)
            
Newcol_names = ["vdo_filename","Wav_file" ,"Text"] 
allSpeechTextframe = pd.DataFrame.from_records(CSV_dict, columns=Newcol_names)
storePath = SpeechText 
finalpath=os.path.join(storePath,"allSpeechToTextframe.csv")  
print(finalpath)
allSpeechTextframe.to_csv(finalpath, sep=',' , index=False)

C:\Users\Administrator\Group-23-InVDO/FullAudio\VDO-AD-105.wav
this is electricity this is a power plant this is Timber cold to me
C:\Users\Administrator\Group-23-InVDO/FullAudio\VDO-AD-200.wav
enamel is the strong white outer layer of your tooth surface the thing that's really important to dentist is to make sure that that enamels things strong and resilient for a lifetime the more that we can strengthen and reharden that to surface The Wider their patients either going to be Sinister going to really want to recommend pronamel strong and bright it help to strengthen Andre hard in the enamel it also has same lifting action it's going to give their patients the protection that they need and The Wider seats live they want
C:\Users\Administrator\Group-23-InVDO\SpeechTextn\allSpeechToTextframe.csv
